# 2024 Winter Introduction to Deep Learning
### Based on Prof. Oh's Youtube Lecture
https://youtube.com/playlist?list=PLvbUC2Zh5oJvByu9KL82bswYT2IKf0K1M

> Assignment #9


*   Youtube Lecture #36-40
*   Written by Seungeun Lee

## 1. Vanilla GAN
*     Reference. https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/03-advanced/generative_adversarial_network/main.py, https://github.com/Yangyangii/pytorch-practice/blob/master/GAN.ipynb, https://arxiv.org/pdf/1406.2661

In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
import numpy as np
import datetime
import os
import sys
from matplotlib.pyplot import imshow, imsave
%matplotlib inline

In [2]:
MODEL_NAME = 'GAN'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def get_sample_image(G, n_noise):
    """
        save sample 100 images
    """
    z = torch.randn(100, n_noise).to(DEVICE)
    y_hat = G(z).view(100, 28, 28) # (100, 28, 28)
    result = y_hat.cpu().data.numpy()
    img = np.zeros([280, 280])
    for j in range(10):
        img[j*28:(j+1)*28] = np.concatenate([x for x in result[j*10:(j+1)*10]], axis=-1)
    return img

# Input: Generator and noise
# Output: Generated Image

In [4]:
class Discriminator(nn.Module):
    """
        Simple Discriminator w/ MLP
    """
    def __init__(self, input_size=784, num_classes=1):
        super(Discriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, num_classes),
            nn.Sigmoid(),
        )

    def forward(self, x):
        y_ = x.view(x.size(0), -1)
        y_ = self.layer(y_)
        return y_

# Discriminator = Classification model

In [5]:
class Generator(nn.Module):
    """
        Simple Generator w/ MLP
    """
    def __init__(self, input_size=100, num_classes=784):
        super(Generator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, num_classes),
            nn.Tanh()
        )

    def forward(self, x):
        y_ = self.layer(x)
        y_ = y_.view(x.size(0), 1, 28, 28)
        return y_

# Discriminator = Generating model

In [6]:
n_noise = 100

In [7]:
D = Discriminator().to(DEVICE)
G = Generator(n_noise).to(DEVICE)

In [8]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

In [9]:
mnist = datasets.MNIST(root='../data/', train=True, transform=transform, download=True)
# MNIST for real data

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 3684181.77it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 131991.90it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:06<00:00, 240183.49it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 5771138.68it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [10]:
batch_size = 64
data_loader = DataLoader(dataset=mnist, batch_size=batch_size, shuffle=True, drop_last=True)
# data_loader = MNIST Data for real

In [11]:
criterion = nn.BCELoss()
D_opt = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
G_opt = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Loss function and Optimizer

In [12]:
max_epoch = 50 # need more than 10 epochs for training generator
step = 0
n_critic = 1 # for training more k steps about Discriminator

# Hyperparameters

In [13]:
D_labels = torch.ones(batch_size, 1).to(DEVICE) # Discriminator Label to real
D_fakes = torch.zeros(batch_size, 1).to(DEVICE) # Discriminator Label to fake

# 0=fake / 1=real

In [14]:
if not os.path.exists('samples'):
    os.makedirs('samples')

In [ ]:
for epoch in range(max_epoch):
    for idx, (images, _) in enumerate(data_loader):
        # Training Discriminator with MNIST
        x = images.to(DEVICE)
        x_outputs = D(x)
        D_x_loss = criterion(x_outputs, D_labels)

        z = torch.randn(batch_size, n_noise).to(DEVICE)
        z_outputs = D(G(z))
        D_z_loss = criterion(z_outputs, D_fakes)
        D_loss = D_x_loss + D_z_loss

        D.zero_grad()
        D_loss.backward()
        D_opt.step()

        if step % n_critic == 0:
            # Training Generator
            z = torch.randn(batch_size, n_noise).to(DEVICE)
            z_outputs = D(G(z))
            G_loss = criterion(z_outputs, D_labels)

            G.zero_grad()
            G_loss.backward()
            G_opt.step()

        if step % 500 == 0:
            print('Epoch: {}/{}, Step: {}, D Loss: {}, G Loss: {}'.format(epoch, max_epoch, step, D_loss.item(), G_loss.item()))

        if step % 1000 == 0:
            G.eval()
            img = get_sample_image(G, n_noise)
            imsave('samples/{}_step{}.jpg'.format(MODEL_NAME, str(step).zfill(3)), img, cmap='gray')
            G.train()
        step += 1

Epoch: 0/50, Step: 0, D Loss: 1.3920471668243408, G Loss: 0.7149672508239746
Epoch: 0/50, Step: 500, D Loss: 1.3099281787872314, G Loss: 1.0041247606277466
Epoch: 1/50, Step: 1000, D Loss: 1.3086402416229248, G Loss: 1.2329256534576416
Epoch: 1/50, Step: 1500, D Loss: 1.4244091510772705, G Loss: 0.3975638747215271
Epoch: 2/50, Step: 2000, D Loss: 1.2298365831375122, G Loss: 0.6444817781448364
Epoch: 2/50, Step: 2500, D Loss: 1.1988779306411743, G Loss: 0.7670705914497375
Epoch: 3/50, Step: 3000, D Loss: 1.2167869806289673, G Loss: 1.0016405582427979
Epoch: 3/50, Step: 3500, D Loss: 1.172280192375183, G Loss: 1.3859493732452393
Epoch: 4/50, Step: 4000, D Loss: 1.3067710399627686, G Loss: 1.3210866451263428
Epoch: 4/50, Step: 4500, D Loss: 1.291306972503662, G Loss: 0.8495550751686096
Epoch: 5/50, Step: 5000, D Loss: 1.2255704402923584, G Loss: 1.236320972442627
Epoch: 5/50, Step: 5500, D Loss: 1.3113819360733032, G Loss: 0.871572732925415
Epoch: 6/50, Step: 6000, D Loss: 1.2562173604965

In [ ]:
# generation to image
G.eval()
imshow(get_sample_image(G, n_noise), cmap='gray')

In [ ]:
# Saving params.
torch.save(D.state_dict(), 'D.pkl')
torch.save(G.state_dict(), 'G.pkl')

### Question 1. Explain the code of the '1. Vanilla GAN' section "briefly".

In [ ]:
# 2가지 모델 Generator, Discriminator가 존재
# MNIST를 real data로 삼아 Discriminator를 학습
# Generator는 Discriminator를 속이도록 학습

# 학습 1000step마다 get_sample_image 함수를 이용해
# Generator로 이미지를 생성해보고 저장한다

## Congratulations on finishing the "Introduction the Deep Learning"!!!
## Now you're a deep learning expert too!

#### After this session, if you're interested in further study, I recommend exploring the following papers and codes:

(1) [Computer Vision] U-GAT-IT, Vision Transformer, (Mask, Fast, Faster) R-CNN, YOLO, Detectron(2), nnUnet, MobileNet, Neural Style Transfer

(2) [Natural Language Processing] KoBERT, BERT, KoBART, BART, ELECTRA

(3) [Audio] ResNeXt, ResMax, AASIST

(4) [Learning Technique] Curriculum Learning, Moco Encoder, Contrastive Learning, Student-teacher Model, Knowledge Distillation, Meta Learning

(5) [Machine Learning for Tabular data Analysis] SVM, LR, XGBoost, LightGBM, CatBoost, TabNet, AutoML (optuna)

Feel free to delve into these topics to deepen your understanding in various areas of AI! If you have any questions about specific topics or need more information, feel free to ask.

### The End.

##### Please upload your Colab file @Github https://github.com/duneag2/intro-dl/tree/main/Assignment8

*   First, make your folder by your name (e.g. seungeun)
*   Then upload your "Jupyter Notebook" file under that directory

###### Need Help?



*   Please refer to this link https://yeko90.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-colab%EC%BD%94%EB%9E%A9%EC%97%90%EC%84%9C-%EC%95%95%EC%B6%95%ED%8C%8C%EC%9D%BC-%ED%92%80%EA%B8%B0 OR
*   Just save your Jupyter Notebook (.ipynb) file in here (colab) and upload via 'Add file' - 'Upload files' https://nthree.tistory.com/60